# **STATS**

In [7]:
# @title Default title text
# Install ipywidgets if needed (uncomment the next two lines if running in an environment without ipywidgets)
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension

import ipywidgets as widgets
from ipywidgets import VBox, HBox, Tab, Layout
import numpy as np
from scipy.stats import norm, binom, poisson, expon, uniform, geom
import math
from IPython.display import display

####################################
# Distribution Calculator UI Section
####################################

# Distribution selection widget
dist_dropdown = widgets.Dropdown(
    options=[
        ("Normal Distribution", "normal"),
        ("Binomial Distribution", "binomial"),
        ("Poisson Distribution", "poisson"),
        ("Exponential Distribution", "exponential"),
        ("Uniform Distribution", "uniform"),
        ("Geometric Distribution", "geometric")
    ],
    description="Distribution:",
    style={'description_width': 'initial'}
)

# Function type widget: Choose which function to compute
func_dropdown = widgets.Dropdown(
    options=[("CDF", "cdf"), ("1 - CDF", "one_minus_cdf"), ("PDF/PMF", "pdf")],
    description="Function Type:",
    style={'description_width': 'initial'}
)

# Container for parameter widgets (will be updated based on distribution)
param_box = VBox([])

# For normal distribution extra input mode
# (This widget will only be used when dist == "normal")
normal_mode = widgets.ToggleButtons(
    options=["Raw", "Z-score", "Percentile"],
    description="Normal Input Mode:",
    style={'description_width': 'initial'},
    layout=Layout(width='400px')
)
# Container for normal-specific parameter inputs
normal_param_box = VBox([], layout=Layout(margin='10px 0px'))

def update_normal_params(*args):
    """Update the parameter widgets for Normal distribution based on the chosen input mode."""
    mode = normal_mode.value
    if mode == "Raw":
        mean_widget = widgets.FloatText(description="Mean (μ):", value=0.0, layout=Layout(width='300px'))
        std_widget = widgets.FloatText(description="Std Dev (σ):", value=1.0, layout=Layout(width='300px'))
        x_widget = widgets.FloatText(description="x:", value=0.0, layout=Layout(width='300px'))
        normal_param_box.children = [mean_widget, std_widget, x_widget]
    elif mode == "Z-score":
        z_widget = widgets.FloatText(description="z-score:", value=0.0, layout=Layout(width='300px'))
        normal_param_box.children = [z_widget]
    elif mode == "Percentile":
        perc_widget = widgets.BoundedFloatText(description="Percentile:", min=0.0, max=1.0, step=0.01, value=0.5, layout=Layout(width='300px'))
        normal_param_box.children = [perc_widget]

# Update normal-specific parameters when mode changes
normal_mode.observe(update_normal_params, names="value")
update_normal_params()  # initialize with default ("Raw")

def update_params(*args):
    """Update parameter widgets based on the chosen distribution."""
    dist = dist_dropdown.value
    widgets_list = []

    if dist == "normal":
        # For normal distribution, include the extra input mode toggle and container
        widgets_list = [normal_mode, normal_param_box]
    elif dist == "binomial":
        n_widget = widgets.IntText(description="n (trials):", value=10, layout=Layout(width='300px'))
        p_widget = widgets.BoundedFloatText(description="p (prob):", min=0.0, max=1.0, step=0.01, value=0.5, layout=Layout(width='300px'))
        x_widget = widgets.IntText(description="x (successes):", value=5, layout=Layout(width='300px'), style={'description_width': '150px'})
        widgets_list = [n_widget, p_widget, x_widget]
    elif dist == "poisson":
        lam_widget = widgets.FloatText(description="λ (rate):", value=3.0, layout=Layout(width='300px'))
        x_widget = widgets.IntText(description="x:", value=2, layout=Layout(width='300px'))
        widgets_list = [lam_widget, x_widget]
    elif dist == "exponential":
        lam_widget = widgets.FloatText(description="λ (rate):", value=1.0, layout=Layout(width='300px'))
        x_widget = widgets.FloatText(description="x:", value=0.0, layout=Layout(width='300px'))
        widgets_list = [lam_widget, x_widget]
    elif dist == "uniform":
        lower_widget = widgets.FloatText(description="Lower bound (a):", value=0.0, layout=Layout(width='300px'), style={'description_width': '150px'})
        upper_widget = widgets.FloatText(description="Upper bound (b):", value=1.0, layout=Layout(width='300px'), style={'description_width': '150px'})
        x_widget = widgets.FloatText(description="x:", value=0.5, layout=Layout(width='300px'))
        widgets_list = [lower_widget, upper_widget, x_widget]
    elif dist == "geometric":
        p_widget = widgets.BoundedFloatText(description="p (prob):", min=0.0, max=1.0, step=0.01, value=0.5, layout=Layout(width='300px'))
        x_widget = widgets.IntText(description="x (trial number):", value=1, layout=Layout(width='300px'), style={'description_width': '150px'})
        widgets_list = [p_widget, x_widget]

    param_box.children = widgets_list

# Update parameters initially and when distribution selection changes
update_params()
dist_dropdown.observe(update_params, names="value")

# Output widget for displaying the result of the distribution calculation
dist_output = widgets.Output()

def calc_distribution(b):
    dist_output.clear_output()
    with dist_output:
        try:
            dist = dist_dropdown.value
            func_type = func_dropdown.value
            result = None

            if dist == "normal":
                # For normal distribution, check the input mode
                mode = normal_mode.value
                if mode == "Raw":
                    # Expecting [Mean, Std Dev, x]
                    mean = normal_param_box.children[0].value
                    std = normal_param_box.children[1].value
                    x_val = normal_param_box.children[2].value
                    d = norm(loc=mean, scale=std)
                    if func_type == "cdf":
                        result = d.cdf(x_val)
                    elif func_type == "one_minus_cdf":
                        result = 1 - d.cdf(x_val)
                    else:
                        result = d.pdf(x_val)
                    print(f"Result: {result}")
                elif mode == "Z-score":
                    # Expecting [z-score]
                    z_val = normal_param_box.children[0].value
                    d = norm(0, 1)
                    if func_type == "cdf":
                        result = d.cdf(z_val)
                    elif func_type == "one_minus_cdf":
                        result = 1 - d.cdf(z_val)
                    else:
                        result = d.pdf(z_val)
                    print(f"Result (Standard Normal at z={z_val}): {result}")
                elif mode == "Percentile":
                    # Expecting [Percentile] to convert to z-score
                    perc = normal_param_box.children[0].value
                    z_val = norm.ppf(perc)
                    print(f"Percentile {perc} corresponds to z-score: {z_val}")
            elif dist == "binomial":
                n = param_box.children[0].value
                p = param_box.children[1].value
                x = param_box.children[2].value
                d = binom(n, p)
                if func_type == "cdf":
                    result = d.cdf(x)
                elif func_type == "one_minus_cdf":
                    result = 1 - d.cdf(x)
                else:
                    result = d.pmf(x)
                print(f"Result: {result}")
            elif dist == "poisson":
                lam = param_box.children[0].value
                x = param_box.children[1].value
                d = poisson(lam)
                if func_type == "cdf":
                    result = d.cdf(x)
                elif func_type == "one_minus_cdf":
                    result = 1 - d.cdf(x)
                else:
                    result = d.pmf(x)
                print(f"Result: {result}")
            elif dist == "exponential":
                lam = param_box.children[0].value
                x = param_box.children[1].value
                d = expon(scale=1/lam)
                if func_type == "cdf":
                    result = d.cdf(x)
                elif func_type == "one_minus_cdf":
                    result = 1 - d.cdf(x)
                else:
                    result = d.pdf(x)
                print(f"Result: {result}")
            elif dist == "uniform":
                a = param_box.children[0].value
                b_val = param_box.children[1].value
                if b_val <= a:
                    raise ValueError("Upper bound must be greater than lower bound.")
                x = param_box.children[2].value
                d = uniform(loc=a, scale=b_val - a)
                if func_type == "cdf":
                    result = d.cdf(x)
                elif func_type == "one_minus_cdf":
                    result = 1 - d.cdf(x)
                else:
                    result = d.pdf(x)
                print(f"Result: {result}")
            elif dist == "geometric":
                p = param_box.children[0].value
                x = param_box.children[1].value
                d = geom(p)
                if func_type == "cdf":
                    result = d.cdf(x)
                elif func_type == "one_minus_cdf":
                    result = 1 - d.cdf(x)
                else:
                    result = d.pmf(x)
                print(f"Result: {result}")
        except Exception as e:
            print(f"Error: {e}")

# Button to trigger distribution calculation
calc_button = widgets.Button(description="Calculate", button_style="success", layout=Layout(width='150px'))
calc_button.on_click(calc_distribution)

# Assemble Distribution Calculator UI
dist_calculator_ui = VBox([
    dist_dropdown,
    func_dropdown,
    param_box,
    calc_button,
    dist_output
], layout=Layout(margin='10px 0px'))

####################################
# Conversion Calculator UI Section
####################################

# Conversion category: either convert a single value or convert parameters.
conv_category = widgets.RadioButtons(
    options=["Value Conversion", "Parameter Conversion"],
    description="Conversion Category:",
    style={'description_width': 'initial'},
    layout=Layout(width='400px')
)

# Container to hold the inner conversion UI (which changes based on category)
conv_inner_box = VBox([], layout=Layout(margin='10px 0px'))

# --- For Value Conversion ---
conv_radio_value = widgets.RadioButtons(
    options=[("Normal to Lognormal", "normal_to_lognormal"), ("Lognormal to Normal", "lognormal_to_normal")],
    description="Value Conversion:",
    style={'description_width': 'initial'},
    layout=Layout(width='400px')
)
conv_value_input = widgets.FloatText(
    description="Value:",
    value=0.0,
    layout=Layout(width='300px')
)
value_conv_ui = VBox([conv_radio_value, conv_value_input], layout=Layout(margin='10px 0px'))

# --- For Parameter Conversion ---
parameter_conv_radio = widgets.RadioButtons(
    options=[("Lognormal to Normal Parameters", "normal_to_lognormal_params"),
             ("Normal to Lognormal Parameters", "lognormal_to_normal_params")],
    description="Parameter Conversion:",
    style={'description_width': 'initial'},
    layout=Layout(width='400px')
)
# Containers for the parameter inputs with fixed width and margin
normal_param_inputs = VBox([
    widgets.FloatText(description="Lognormal Mean (m):", value=0.0, layout=Layout(width='300px'), style={'description_width': '150px'}),
    widgets.FloatText(description="Lognormal Std Dev:", value=1.0, layout=Layout(width='300px'), style={'description_width': '150px'})
], layout=Layout(margin='10px 0px'))
lognormal_param_inputs = VBox([
    widgets.FloatText(description="Normal Mean (μ):", value=1.0, layout=Layout(width='300px'), style={'description_width': '150px'}),
    widgets.FloatText(description="Normal Variance:", value=0.5, layout=Layout(width='300px'), style={'description_width': '150px'})
], layout=Layout(margin='10px 0px'))
# Container that will hold one of the above based on the radio selection.
param_conv_input_box = VBox([], layout=Layout(margin='10px 0px'))

def update_param_conv_inputs(*args):
    """Update parameter conversion inputs based on chosen conversion type."""
    if parameter_conv_radio.value == "normal_to_lognormal_params":
        param_conv_input_box.children = [normal_param_inputs]
    else:
        param_conv_input_box.children = [lognormal_param_inputs]

parameter_conv_radio.observe(update_param_conv_inputs, names="value")
update_param_conv_inputs()

parameter_conv_ui = VBox([parameter_conv_radio, param_conv_input_box], layout=Layout(margin='10px 0px'))

def update_conv_inner(*args):
    """Update the inner conversion UI based on conversion category."""
    if conv_category.value == "Value Conversion":
        conv_inner_box.children = [value_conv_ui]
    else:
        conv_inner_box.children = [parameter_conv_ui]

conv_category.observe(update_conv_inner, names="value")
update_conv_inner()

# Output widget for conversion results
conv_output = widgets.Output()

# Button for performing conversion
conv_button = widgets.Button(description="Convert", button_style="info", layout=Layout(width='150px'))

def perform_conversion(b):
    conv_output.clear_output()
    with conv_output:
        try:
            if conv_category.value == "Value Conversion":
                val = conv_value_input.value
                if conv_radio_value.value == "normal_to_lognormal":
                    # Single value conversion: normal to lognormal using exp.
                    result = math.exp(val)
                    print(f"exp({val}) = {result}")
                else:
                    # Lognormal to normal using log.
                    if val <= 0:
                        raise ValueError("Value must be positive for log conversion.")
                    result = math.log(val)
                    print(f"log({val}) = {result}")
            else:
                # Parameter Conversion
                if parameter_conv_radio.value == "normal_to_lognormal_params":
                    mu = normal_param_inputs.children[0].value
                    sigma = normal_param_inputs.children[1].value
                    lognorm_mean = math.exp(mu + sigma**2 / 2)
                    lognorm_variance = (math.exp(sigma**2) - 1) * math.exp(2*mu + sigma**2)
                    print(f"Given Lognormal parameters: μ = {mu}, σ = {sigma}")
                    print(f"Equivalent Normal Mean: {lognorm_mean}")
                    print(f"Equivalent Normal Variance: {lognorm_variance}")
                else:
                    m = lognormal_param_inputs.children[0].value
                    v = lognormal_param_inputs.children[1].value
                    if m <= 0:
                        raise ValueError("Lognormal mean must be positive.")
                    sigma2 = math.log(1 + v/(m**2))
                    sigma = math.sqrt(sigma2)
                    mu = math.log(m) - sigma2/2
                    print(f"Given Normal parameters: Mean = {m}, Variance = {v}")
                    print(f"Lognormal Mean: {mu}")
                    print(f"Lognormal Std Dev: {sigma}")
        except Exception as e:
            print(f"Error: {e}")

conv_button.on_click(perform_conversion)

# Assemble Conversion Calculator UI
conv_calculator_ui = VBox([
    conv_category,
    conv_inner_box,
    conv_button,
    conv_output
], layout=Layout(margin='10px 0px'))

####################################
# Create Tabs for the Two Calculators
####################################

tab = Tab(children=[dist_calculator_ui, conv_calculator_ui])
tab.set_title(0, "Distributions")
tab.set_title(1, "Conversion")
display(tab)


# **Trig/Log**

In [3]:
# @title Default title text
# Install ipywidgets if needed (uncomment if necessary)
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension

import ipywidgets as widgets
from ipywidgets import VBox, HBox, Tab, Layout
import math
from IPython.display import display

####################################
# Tab 1: Trigonometric Functions
####################################

# Dropdown for selecting trigonometric function
trig_function_dropdown = widgets.Dropdown(
    options=["sin", "cos", "tan", "cosec", "sec", "cot"],
    description="Function:",
    style={'description_width': '120px'},
    layout=Layout(width='300px')
)

# Radio buttons for angle unit (Radians or Degrees)
angle_unit_radio = widgets.RadioButtons(
    options=["Radians", "Degrees"],
    value="Degrees",
    description="Angle Unit:",
    style={'description_width': '120px'},
    layout=Layout(width='300px')
)

# Input widget for the angle
angle_input = widgets.FloatText(
    description="Angle:",
    value=0.0,
    style={'description_width': '120px'},
    layout=Layout(width='300px')
)

# Button to perform calculation
trig_calc_button = widgets.Button(
    description="Calculate",
    button_style="success",
    layout=Layout(width='150px')
)

# Output widget for trigonometric results
trig_output = widgets.Output()

def calculate_trig(b):
    trig_output.clear_output()
    with trig_output:
        try:
            func = trig_function_dropdown.value
            angle = angle_input.value
            unit = angle_unit_radio.value
            # Convert degrees to radians if needed
            if unit == "Degrees":
                angle = math.radians(angle)
            # Compute the desired trigonometric function
            if func == "sin":
                result = math.sin(angle)
            elif func == "cos":
                result = math.cos(angle)
            elif func == "tan":
                result = math.tan(angle)
            elif func == "cosec":
                s = math.sin(angle)
                if s == 0:
                    raise ValueError("Cosecant undefined when sin(angle)=0")
                result = 1/s
            elif func == "sec":
                c = math.cos(angle)
                if c == 0:
                    raise ValueError("Secant undefined when cos(angle)=0")
                result = 1/c
            elif func == "cot":
                t = math.tan(angle)
                if t == 0:
                    raise ValueError("Cotangent undefined when tan(angle)=0")
                result = 1/t
            print(f"{func}({angle_input.value} {unit}) = {result}")
        except Exception as e:
            print("Error:", e)

trig_calc_button.on_click(calculate_trig)

trig_ui = VBox([
    trig_function_dropdown,
    angle_unit_radio,
    angle_input,
    trig_calc_button,
    trig_output
], layout=Layout(margin='10px 0px'))

####################################
# Tab 2: Logarithms
####################################

# Dropdown for selecting logarithm type
log_type_dropdown = widgets.Dropdown(
    options=[("log base 10", "log10"), ("Natural log (ln)", "ln"), ("Log with custom base", "log_custom")],
    description="Log Type:",
    style={'description_width': '150px'},
    layout=Layout(width='350px')
)

# Input widget for the value to take log of
log_value_input = widgets.FloatText(
    description="Value:",
    value=1.0,
    style={'description_width': '150px'},
    layout=Layout(width='300px')
)

# Input widget for custom base (only visible if custom base is selected)
log_custom_base_input = widgets.FloatText(
    description="Base:",
    value=10.0,
    style={'description_width': '150px'},
    layout=Layout(width='300px')
)

# Function to update visibility of the custom base input
def update_log_inputs(*args):
    if log_type_dropdown.value == "log_custom":
        log_custom_base_input.layout.display = 'block'
    else:
        log_custom_base_input.layout.display = 'none'

log_type_dropdown.observe(update_log_inputs, names="value")
update_log_inputs()

# Button for log calculation
log_calc_button = widgets.Button(
    description="Calculate",
    button_style="success",
    layout=Layout(width='150px')
)

# Output widget for logarithm results
log_output = widgets.Output()

def calculate_log(b):
    log_output.clear_output()
    with log_output:
        try:
            log_type = log_type_dropdown.value
            x = log_value_input.value
            if x <= 0:
                raise ValueError("Logarithm undefined for non-positive values")
            if log_type == "log10":
                result = math.log10(x)
                print(f"log10({x}) = {result}")
            elif log_type == "ln":
                result = math.log(x)
                print(f"ln({x}) = {result}")
            else:
                base = log_custom_base_input.value
                if base <= 0 or base == 1:
                    raise ValueError("Base must be positive and not equal to 1")
                result = math.log(x, base)
                print(f"log base {base} of {x} = {result}")
        except Exception as e:
            print("Error:", e)

log_calc_button.on_click(calculate_log)

log_ui = VBox([
    log_type_dropdown,
    log_value_input,
    log_custom_base_input,
    log_calc_button,
    log_output
], layout=Layout(margin='10px 0px'))

####################################
# Tab 3: Inverse Trigonometric Functions
####################################

# Dropdown for selecting inverse trigonometric function
inv_trig_dropdown = widgets.Dropdown(
    options=[("arcsin", "arcsin"), ("arccos", "arccos"), ("arctan", "arctan"),
             ("arccosec", "arccosec"), ("arcsec", "arcsec"), ("arccot", "arccot")],
    description="Inverse Function:",
    style={'description_width': '150px'},
    layout=Layout(width='350px')
)

# Radio buttons for output unit (Radians or Degrees)
inv_angle_unit_radio = widgets.RadioButtons(
    options=["Radians", "Degrees"],
    value="Degrees",
    description="Result Unit:",
    style={'description_width': '150px'},
    layout=Layout(width='300px')
)

# Input widget for the value for which to compute the inverse function
inv_value_input = widgets.FloatText(
    description="Value:",
    value=0.0,
    style={'description_width': '150px'},
    layout=Layout(width='300px')
)

# Button for inverse trig calculation
inv_trig_calc_button = widgets.Button(
    description="Calculate",
    button_style="success",
    layout=Layout(width='150px')
)

# Output widget for inverse trig results
inv_trig_output = widgets.Output()

def calculate_inv_trig(b):
    inv_trig_output.clear_output()
    with inv_trig_output:
        try:
            func = inv_trig_dropdown.value
            x = inv_value_input.value
            result = None
            if func == "arcsin":
                if x < -1 or x > 1:
                    raise ValueError("arcsin undefined for |x|>1")
                result = math.asin(x)
            elif func == "arccos":
                if x < -1 or x > 1:
                    raise ValueError("arccos undefined for |x|>1")
                result = math.acos(x)
            elif func == "arctan":
                result = math.atan(x)
            elif func == "arccosec":
                if x == 0 or abs(x) < 1:
                    raise ValueError("arccosec undefined for |x|<1 or x=0")
                result = math.asin(1/x)
            elif func == "arcsec":
                if x == 0 or abs(x) < 1:
                    raise ValueError("arcsec undefined for |x|<1 or x=0")
                result = math.acos(1/x)
            elif func == "arccot":
                # Using identity: arccot(x) = π/2 - arctan(x)
                result = math.pi/2 - math.atan(x)
            # Convert result to degrees if requested
            if inv_angle_unit_radio.value == "Degrees":
                result = math.degrees(result)
            print(f"{func}({x}) = {result} {inv_angle_unit_radio.value}")
        except Exception as e:
            print("Error:", e)

inv_trig_calc_button.on_click(calculate_inv_trig)

inv_trig_ui = VBox([
    inv_trig_dropdown,
    inv_angle_unit_radio,
    inv_value_input,
    inv_trig_calc_button,
    inv_trig_output
], layout=Layout(margin='10px 0px'))

####################################
# Assemble Tabs
####################################

tab = Tab(children=[trig_ui, log_ui, inv_trig_ui])
tab.set_title(0, "Trigonometry")
tab.set_title(1, "Logarithms")
tab.set_title(2, "Inverse Trigonometry")
display(tab)


# **Matrix, Equation solver, Integral, Maxima,Minima**

In [ ]:
# @title Default title text
# Install ipywidgets if needed (uncomment if necessary)
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension

import ipywidgets as widgets
from ipywidgets import VBox, HBox, Tab, Layout
import numpy as np
import sympy as sp
import math
from IPython.display import display

# Helper: Check if an expression contains any trigonometric functions
def contains_trig(expr):
    trig_funcs = [sp.sin, sp.cos, sp.tan, sp.csc, sp.sec, sp.cot]
    return any(expr.has(func) for func in trig_funcs)

####################################
# MATRIX OPERATIONS TAB
####################################

matrix_op_dropdown = widgets.Dropdown(
    options=["Addition", "Multiplication", "Inverse", "Determinant"],
    description="Matrix Operation:",
    style={'description_width': '150px'},
    layout=Layout(width='400px')
)

matrix_textarea_A = widgets.Textarea(
    value="1,2,3;4,5,6",
    description="Matrix A:",
    layout=Layout(width='400px', height='100px'),
    style={'description_width': '100px'}
)
matrix_textarea_B = widgets.Textarea(
    value="7,8,9;10,11,12",
    description="Matrix B:",
    layout=Layout(width='400px', height='100px'),
    style={'description_width': '100px'}
)
matrix_textarea = widgets.Textarea(
    value="1,2,3;4,5,6",
    description="Matrix:",
    layout=Layout(width='400px', height='100px'),
    style={'description_width': '100px'}
)

matrix_input_box = VBox()

def update_matrix_inputs(*args):
    op = matrix_op_dropdown.value
    if op in ["Addition", "Multiplication"]:
        matrix_input_box.children = [matrix_textarea_A, matrix_textarea_B]
    else:
        matrix_input_box.children = [matrix_textarea]

update_matrix_inputs()
matrix_op_dropdown.observe(update_matrix_inputs, names="value")

matrix_output = widgets.Output()

def parse_matrix(text):
    try:
        rows = text.strip().split(";")
        matrix = [list(map(float, row.split(","))) for row in rows]
        return np.array(matrix)
    except Exception as e:
        raise ValueError("Invalid matrix format. Use rows separated by ';' and elements by ','.")

def calculate_matrix_operation(b):
    matrix_output.clear_output()
    with matrix_output:
        try:
            op = matrix_op_dropdown.value
            if op in ["Addition", "Multiplication"]:
                A = parse_matrix(matrix_textarea_A.value)
                B = parse_matrix(matrix_textarea_B.value)
                result = A + B if op == "Addition" else np.matmul(A, B)
            else:
                M = parse_matrix(matrix_textarea.value)
                result = np.linalg.inv(M) if op == "Inverse" else np.linalg.det(M)
            print("Result:")
            print(result)
        except Exception as e:
            print("Error:", e)

matrix_calc_button = widgets.Button(
    description="Calculate",
    button_style="success",
    layout=Layout(width='150px')
)
matrix_calc_button.on_click(calculate_matrix_operation)

matrix_tab_ui = VBox([
    matrix_op_dropdown,
    matrix_input_box,
    matrix_calc_button,
    matrix_output
], layout=Layout(margin='10px 0px'))

####################################
# EQUATION SOLVER TAB
####################################

equation_solver_dropdown = widgets.Dropdown(
    options=["2 Variables", "3 Variables", "Quadratic Equation", "Cubic Equation"],
    description="Equation Type:",
    style={'description_width': '150px'},
    layout=Layout(width='400px')
)

# 2 Variables: a1*x + b1*y = c1 and a2*x + b2*y = c2
a1_2 = widgets.FloatText(description="a1:", layout=Layout(width='150px'))
b1_2 = widgets.FloatText(description="b1:", layout=Layout(width='150px'))
c1_2 = widgets.FloatText(description="c1:", layout=Layout(width='150px'))
a2_2 = widgets.FloatText(description="a2:", layout=Layout(width='150px'))
b2_2 = widgets.FloatText(description="b2:", layout=Layout(width='150px'))
c2_2 = widgets.FloatText(description="c2:", layout=Layout(width='150px'))
two_vars_box = VBox([
    HBox([a1_2, b1_2, c1_2]),
    HBox([a2_2, b2_2, c2_2])
])

# 3 Variables: a1*x + b1*y + c1*z = d1, etc.
a1_3 = widgets.FloatText(description="a1:", layout=Layout(width='150px'))
b1_3 = widgets.FloatText(description="b1:", layout=Layout(width='150px'))
c1_3 = widgets.FloatText(description="c1:", layout=Layout(width='150px'))
d1_3 = widgets.FloatText(description="d1:", layout=Layout(width='150px'))
a2_3 = widgets.FloatText(description="a2:", layout=Layout(width='150px'))
b2_3 = widgets.FloatText(description="b2:", layout=Layout(width='150px'))
c2_3 = widgets.FloatText(description="c2:", layout=Layout(width='150px'))
d2_3 = widgets.FloatText(description="d2:", layout=Layout(width='150px'))
a3_3 = widgets.FloatText(description="a3:", layout=Layout(width='150px'))
b3_3 = widgets.FloatText(description="b3:", layout=Layout(width='150px'))
c3_3 = widgets.FloatText(description="c3:", layout=Layout(width='150px'))
d3_3 = widgets.FloatText(description="d3:", layout=Layout(width='150px'))
three_vars_box = VBox([
    HBox([a1_3, b1_3, c1_3, d1_3]),
    HBox([a2_3, b2_3, c2_3, d2_3]),
    HBox([a3_3, b3_3, c3_3, d3_3])
])

# Quadratic Equation: a*x^2 + b*x + c = 0
a_quad = widgets.FloatText(description="a:", layout=Layout(width='150px'))
b_quad = widgets.FloatText(description="b:", layout=Layout(width='150px'))
c_quad = widgets.FloatText(description="c:", layout=Layout(width='150px'))
quad_box = HBox([a_quad, b_quad, c_quad])

# Cubic Equation: a*x^3 + b*x^2 + c*x + d = 0
a_cubic = widgets.FloatText(description="a:", layout=Layout(width='150px'))
b_cubic = widgets.FloatText(description="b:", layout=Layout(width='150px'))
c_cubic = widgets.FloatText(description="c:", layout=Layout(width='150px'))
d_cubic = widgets.FloatText(description="d:", layout=Layout(width='150px'))
cubic_box = HBox([a_cubic, b_cubic, c_cubic, d_cubic])

equation_input_box = VBox()

def update_equation_inputs(*args):
    eq_type = equation_solver_dropdown.value
    if eq_type == "2 Variables":
        equation_input_box.children = [two_vars_box]
    elif eq_type == "3 Variables":
        equation_input_box.children = [three_vars_box]
    elif eq_type == "Quadratic Equation":
        equation_input_box.children = [quad_box]
    else:
        equation_input_box.children = [cubic_box]

update_equation_inputs()
equation_solver_dropdown.observe(update_equation_inputs, names="value")

eq_output = widgets.Output()

def solve_equation(b):
    eq_output.clear_output()
    with eq_output:
        x, y, z = sp.symbols('x y z')
        eq_type = equation_solver_dropdown.value
        try:
            if eq_type == "2 Variables":
                eq1 = sp.Eq(a1_2.value*x + b1_2.value*y, c1_2.value)
                eq2 = sp.Eq(a2_2.value*x + b2_2.value*y, c2_2.value)
                sol = sp.solve([eq1, eq2], (x, y))
            elif eq_type == "3 Variables":
                eq1 = sp.Eq(a1_3.value*x + b1_3.value*y + c1_3.value*z, d1_3.value)
                eq2 = sp.Eq(a2_3.value*x + b2_3.value*y + c2_3.value*z, d2_3.value)
                eq3 = sp.Eq(a3_3.value*x + b3_3.value*y + c3_3.value*z, d3_3.value)
                sol = sp.solve([eq1, eq2, eq3], (x, y, z))
            elif eq_type == "Quadratic Equation":
                sol = sp.solve(a_quad.value*x**2 + b_quad.value*x + c_quad.value, x)
            else:
                sol = sp.solve(a_cubic.value*x**3 + b_cubic.value*x**2 + c_cubic.value*x + d_cubic.value, x)
            print("Solution:", sol)
        except Exception as e:
            print("Error:", e)

eq_solve_button = widgets.Button(
    description="Solve",
    button_style="success",
    layout=Layout(width='150px')
)
eq_solve_button.on_click(solve_equation)

eq_solver_ui = VBox([
    equation_solver_dropdown,
    equation_input_box,
    eq_solve_button,
    eq_output
], layout=Layout(margin='10px 0px'))

####################################
# INTEGRAL CALCULATOR TAB
####################################

integral_function = widgets.Textarea(
    value="x**2",
    description="f(x):",
    layout=Layout(width='400px', height='100px'),
    style={'description_width': '80px'}
)

def append_to_integral(btn):
    integral_function.value += btn.description

integral_buttons = [
    widgets.Button(description="sin(", layout=Layout(width='60px')),
    widgets.Button(description="cos(", layout=Layout(width='60px')),
    widgets.Button(description="tan(", layout=Layout(width='60px')),
    widgets.Button(description="exp(", layout=Layout(width='60px')),
    widgets.Button(description="log(", layout=Layout(width='60px')),
    widgets.Button(description="sqrt(", layout=Layout(width='60px')),
    widgets.Button(description="^", layout=Layout(width='40px'))
]
for btn in integral_buttons:
    btn.on_click(append_to_integral)
integral_buttons_box = HBox(integral_buttons)

integral_lower = widgets.FloatText(
    value=0.0,
    description="Lower Limit:",
    layout=Layout(width='200px'),
    style={'description_width': '100px'}
)
integral_upper = widgets.FloatText(
    value=2.0,
    description="Upper Limit:",
    layout=Layout(width='200px'),
    style={'description_width': '100px'}
)

# For integrals, the "angle unit" applies only if the function contains trigonometric functions.
integral_angle_unit = widgets.RadioButtons(
    options=["Degrees", "Radians"],
    value="Degrees",
    description="Angle Unit:",
    layout=Layout(width='200px'),
    style={'description_width': '100px'}
)

integral_calc_button = widgets.Button(
    description="Calculate Integral",
    button_style="success",
    layout=Layout(width='200px')
)
integral_output = widgets.Output()

def calculate_integral(b):
    integral_output.clear_output()
    with integral_output:
        try:
            x = sp.symbols('x')
            expr = sp.sympify(integral_function.value, convert_xor=True)
            # Only substitute x if the expression contains a trig function and the unit is Degrees.
            if integral_angle_unit.value == "Degrees" and contains_trig(expr):
                expr = expr.subs(x, sp.pi/180 * x)
            lower = integral_lower.value
            upper = integral_upper.value
            result = sp.integrate(expr, (x, lower, upper))
            result_numeric = sp.N(result)  # Evaluate to numeric value
            print("Integral:", result_numeric)
        except Exception as e:
            print("Error:", e)

integral_calc_button.on_click(calculate_integral)

integral_ui = VBox([
    integral_function,
    integral_buttons_box,
    HBox([integral_lower, integral_upper]),
    integral_angle_unit,
    integral_calc_button,
    integral_output
], layout=Layout(margin='10px 0px'))

####################################
# MAXIMA/MINIMA CALCULATOR TAB
####################################

extrema_function = widgets.Textarea(
    value="x**2",
    description="f(x):",
    layout=Layout(width='400px', height='100px'),
    style={'description_width': '80px'}
)

def append_to_extrema(btn):
    extrema_function.value += btn.description

ext_buttons = [
    widgets.Button(description="sin(", layout=Layout(width='60px')),
    widgets.Button(description="cos(", layout=Layout(width='60px')),
    widgets.Button(description="tan(", layout=Layout(width='60px')),
    widgets.Button(description="exp(", layout=Layout(width='60px')),
    widgets.Button(description="log(", layout=Layout(width='60px')),
    widgets.Button(description="sqrt(", layout=Layout(width='60px')),
    widgets.Button(description="^", layout=Layout(width='40px'))
]
for btn in ext_buttons:
    btn.on_click(append_to_extrema)
ext_buttons_box = HBox(ext_buttons)

ext_lower = widgets.FloatText(
    value=-50,
    description="Lower Bound:",
    layout=Layout(width='200px'),
    style={'description_width': '100px'}
)
ext_upper = widgets.FloatText(
    value=50,
    description="Upper Bound:",
    layout=Layout(width='200px'),
    style={'description_width': '100px'}
)

ext_angle_unit = widgets.RadioButtons(
    options=["Degrees", "Radians"],
    value="Degrees",
    description="Angle Unit:",
    layout=Layout(width='200px'),
    style={'description_width': '100px'}
)

ext_calc_button = widgets.Button(
    description="Find Extrema",
    button_style="success",
    layout=Layout(width='150px')
)
ext_output = widgets.Output()

def calculate_extrema(b):
    ext_output.clear_output()
    with ext_output:
        try:
            import numpy as np
            # Parse the function using sympy
            x = sp.symbols('x', real=True)
            f_expr = sp.sympify(extrema_function.value, convert_xor=True)
            lower_bound = float(ext_lower.value)
            upper_bound = float(ext_upper.value)

            # Define a numerical function that respects the selected angle unit.
            if (ext_angle_unit.value == "Degrees" and
                any(f_expr.has(func) for func in [sp.sin, sp.cos, sp.tan, sp.csc, sp.sec, sp.cot])):
                f_lambda = sp.lambdify(x, f_expr, "numpy")
                def f_num(deg):
                    return f_lambda(np.deg2rad(deg))
            else:
                f_num = sp.lambdify(x, f_expr, "numpy")

            # Create a dense grid over the interval.
            xs = np.linspace(lower_bound, upper_bound, 10000)
            ys = f_num(xs)

            # Get the indices from the grid.
            min_idx = np.argmin(ys)
            max_idx = np.argmax(ys)

            # Start candidate set with the grid-found extrema and endpoints.
            candidates = [xs[min_idx], xs[max_idx], lower_bound, upper_bound]

            # If using degrees and a trig function, explicitly add common angles.
            if ext_angle_unit.value == "Degrees" and any(f_expr.has(func) for func in
                                                         [sp.sin, sp.cos, sp.tan, sp.csc, sp.sec, sp.cot]):
                for pt in [-90, 0, 90]:
                    if lower_bound <= pt <= upper_bound:
                        candidates.append(pt)

            # Remove duplicates (using np.unique) and evaluate f at each candidate.
            candidate_vals = {}
            for cand in np.unique(candidates):
                try:
                    val = f_num(cand)
                    candidate_vals[cand] = float(val)
                except Exception as e:
                    print("Error evaluating candidate", cand, ":", e)

            if not candidate_vals:
                print("No valid candidates found.")
                return

            min_candidate = min(candidate_vals.items(), key=lambda t: t[1])
            max_candidate = max(candidate_vals.items(), key=lambda t: t[1])
            print("Minimum at x =", min_candidate[0], "with value =", min_candidate[1])
            print("Maximum at x =", max_candidate[0], "with value =", max_candidate[1])
        except Exception as e:
            print("Error:", e)

ext_calc_button.on_click(calculate_extrema)

extrema_ui = VBox([
    extrema_function,
    ext_buttons_box,
    HBox([ext_lower, ext_upper]),
    ext_angle_unit,
    ext_calc_button,
    ext_output
], layout=Layout(margin='10px 0px'))

####################################
# ASSEMBLE ALL TABS
####################################

all_tabs = Tab(children=[matrix_tab_ui, eq_solver_ui, integral_ui, extrema_ui])
all_tabs.set_title(0, "Matrix Ops")
all_tabs.set_title(1, "Eq Solver")
all_tabs.set_title(2, "Integral")
all_tabs.set_title(3, "Extrema")
display(all_tabs)


# **nCr, nPr, root and power, factorial**

In [ ]:
# @title Default title text
import ipywidgets as widgets
from ipywidgets import VBox, HBox, Layout
import math, fractions
from IPython.display import display

# Global state
calc_expr = "0"         # the current expression shown on display
pending_operator = None # one of "power", "nth_root", "nCr", "nPr"
stored_value = None     # the first operand (as string)

# Mapping of binary operators to display symbols.
operator_symbols = {
    "power": "^",
    "nth_root": " nrt ",
    "nCr": " C ",
    "nPr": " P "
}

# The calculator display
calc_display = widgets.Text(
    value="0",
    disabled=True,
    layout=Layout(width="400px", height="40px", font_size="20px", text_align="right")
)

def update_display():
    calc_display.value = str(calc_expr)

def clear_all():
    global calc_expr, pending_operator, stored_value
    calc_expr = "0"
    pending_operator = None
    stored_value = None
    update_display()

def append_to_expr(s):
    global calc_expr
    # If display is "0", replace it unless appending an operator.
    if calc_expr == "0":
        if s.strip() in operator_symbols.values():
            calc_expr += s
        else:
            calc_expr = s
    else:
        calc_expr += s
    update_display()

def append_digit(digit):
    append_to_expr(digit)

def append_decimal():
    global calc_expr, pending_operator
    if pending_operator:
        op_sym = operator_symbols[pending_operator]
        parts = calc_expr.split(op_sym)
        current = parts[-1]
    else:
        current = calc_expr
    if "." not in current:
        append_to_expr(".")

def clear_all_button():
    clear_all()

def set_operator(op):
    global calc_expr, pending_operator, stored_value
    if pending_operator is None:
        stored_value = calc_expr
        pending_operator = op
        append_to_expr(operator_symbols[op])

def calculate_result():
    global calc_expr, pending_operator, stored_value
    if pending_operator is None:
        return
    op_sym = operator_symbols[pending_operator]
    try:
        parts = calc_expr.split(op_sym)
        if len(parts) != 2:
            update_display_error("Error")
            return
        first_str = parts[0].strip()
        second_str = parts[1].strip()
        if first_str == "" or second_str == "":
            update_display_error("Error")
            return
        first_num = float(first_str)
        second_num = float(second_str)
        result = None
        if pending_operator == "power":
            result = first_num ** second_num
        elif pending_operator == "nth_root":
            # For nth root, we assume the first number is the index and the second is the radicand.
            # So 4 nth_root 8 should yield 8^(1/4).
            if first_num == 0:
                update_display_error("Error")
                return
            result = second_num ** (1/first_num)
        elif pending_operator == "nCr":
            result = math.comb(int(first_num), int(second_num))
        elif pending_operator == "nPr":
            result = math.perm(int(first_num), int(second_num))
        else:
            update_display_error("Error")
            return
        calc_expr = str(result)
        pending_operator = None
        stored_value = None
        update_display()
    except Exception as e:
        update_display_error("Error")

def update_display_error(msg):
    global calc_expr, pending_operator, stored_value
    calc_expr = msg
    pending_operator = None
    stored_value = None
    update_display()

def unary_operation(func):
    global calc_expr, pending_operator
    try:
        if pending_operator is not None:
            op_sym = operator_symbols[pending_operator]
            parts = calc_expr.split(op_sym)
            if len(parts) == 2 and parts[1].strip() != "":
                second_num = float(parts[1].strip())
                result = func(second_num)
                calc_expr = parts[0] + op_sym + str(result)
            else:
                update_display_error("Error")
                return
        else:
            result = func(float(calc_expr))
            calc_expr = str(result)
        update_display()
    except Exception as e:
        update_display_error("Error")

def inverse_op():
    unary_operation(lambda x: 1/x if x != 0 else float('inf'))

def square_op():
    unary_operation(lambda x: x**2)

def sqrt_op():
    unary_operation(lambda x: math.sqrt(x) if x >= 0 else float('nan'))

def factorial_op():
    unary_operation(lambda x: math.factorial(int(x)) if x >= 0 and int(x) == x else None)

# Enhanced decimal to fraction: show mixed fraction and normal fraction if improper.
def fraction_op():
    global calc_expr
    try:
        num = float(calc_expr)
        frac = fractions.Fraction(num).limit_denominator()
        # Check if improper fraction (absolute numerator greater than denominator)
        if abs(frac.numerator) > frac.denominator:
            whole = abs(frac.numerator) // frac.denominator
            remainder = abs(frac.numerator) % frac.denominator
            sign = "-" if frac.numerator < 0 else ""
            mixed = f"{sign}{whole} {remainder}/{frac.denominator}" if remainder != 0 else f"{sign}{whole}"
            normal = f"{frac.numerator}/{frac.denominator}"
            result_str = f"{mixed} ({normal})"
        else:
            result_str = f"{frac.numerator}/{frac.denominator}"
        calc_expr = result_str
        update_display()
    except Exception as e:
        update_display_error("Error")

def pi_op():
    global calc_expr
    calc_expr = str(math.pi)
    update_display()

def e_op():
    global calc_expr
    calc_expr = str(math.e)
    update_display()

def operator_power(b):
    set_operator("power")

def operator_nth_root(b):
    set_operator("nth_root")

def operator_nCr(b):
    set_operator("nCr")

def operator_nPr(b):
    set_operator("nPr")

def equals_op(b):
    calculate_result()

def clear_button(b):
    clear_all()

# Digit buttons 0-9
digit_buttons = [widgets.Button(description=str(i), layout=Layout(width="60px", height="40px")) for i in range(10)]
for btn in digit_buttons:
    btn.on_click(lambda b, d=btn.description: append_digit(d))

btn_decimal = widgets.Button(description=".", layout=Layout(width="60px", height="40px"))
btn_decimal.on_click(lambda b: append_decimal())

btn_inverse = widgets.Button(description="1/x", layout=Layout(width="60px", height="40px"))
btn_inverse.on_click(lambda b: inverse_op())

btn_square = widgets.Button(description="x²", layout=Layout(width="60px", height="40px"))
btn_square.on_click(lambda b: square_op())

btn_sqrt = widgets.Button(description="√x", layout=Layout(width="60px", height="40px"))
btn_sqrt.on_click(lambda b: sqrt_op())

btn_frac = widgets.Button(description="Frac", layout=Layout(width="60px", height="40px"))
btn_frac.on_click(lambda b: fraction_op())

btn_fact = widgets.Button(description="x!", layout=Layout(width="60px", height="40px"))
btn_fact.on_click(lambda b: factorial_op())

btn_power = widgets.Button(description="^", layout=Layout(width="60px", height="40px"))
btn_power.on_click(operator_power)

btn_nth_root = widgets.Button(description="nrt", layout=Layout(width="60px", height="40px"))
btn_nth_root.on_click(operator_nth_root)

btn_nCr = widgets.Button(description="C", layout=Layout(width="60px", height="40px"))
btn_nCr.on_click(operator_nCr)

btn_nPr = widgets.Button(description="P", layout=Layout(width="60px", height="40px"))
btn_nPr.on_click(operator_nPr)

btn_equals = widgets.Button(description="=", layout=Layout(width="60px", height="40px"), button_style="success")
btn_equals.on_click(equals_op)

btn_pi = widgets.Button(description="π", layout=Layout(width="60px", height="40px"))
btn_pi.on_click(lambda b: pi_op())

btn_e = widgets.Button(description="e", layout=Layout(width="60px", height="40px"))
btn_e.on_click(lambda b: e_op())

btn_clear = widgets.Button(description="C", layout=Layout(width="60px", height="40px"), button_style="danger")
btn_clear.on_click(clear_button)

# Layout the calculator as a grid resembling a regular calculator.
row1 = HBox([calc_display])
row2 = HBox([digit_buttons[7], digit_buttons[8], digit_buttons[9], btn_power, btn_nth_root])
row3 = HBox([digit_buttons[4], digit_buttons[5], digit_buttons[6], btn_nCr, btn_nPr])
row4 = HBox([digit_buttons[1], digit_buttons[2], digit_buttons[3], btn_inverse, btn_square])
row5 = HBox([digit_buttons[0], btn_decimal, btn_frac, btn_fact])
row6 = HBox([btn_pi, btn_e, btn_sqrt, btn_clear, btn_equals])

calculator_ui = VBox([row1, row2, row3, row4, row5, row6])
display(calculator_ui)


# **Basic calculator**

In [ ]:
# @title Default title text
import math
import ipywidgets as widgets
from IPython.display import display

# --- Global State ---
calc_state = {
    "expression": "",
    "currentFunction": "",
    "result": ""
}

# --- Display Widgets ---
display_style = "text-align:right; font-size:32px; color:white; background:#1a1a1a; padding:10px; border-radius:8px; min-height:45px;"

expression_display = widgets.HTML(value=f"<div style='{display_style}'>{calc_state['expression']}</div>")
result_display = widgets.HTML(value=f"<div style='{display_style}'>{calc_state['result']}</div>")

def update_display():
    expr = f"{calc_state['currentFunction']}({calc_state['expression']})" if calc_state["currentFunction"] else calc_state["expression"]
    expression_display.value = f"<div style='{display_style}'>{expr if expr else '&nbsp;'}</div>"
    result_display.value = f"<div style='{display_style}'>{calc_state['result'] if calc_state['result'] else '&nbsp;'}</div>"

# --- Calculator Functions ---
def append_to_expr(val):
    calc_state["expression"] += val
    update_display()

def clear_calc(b=None):
    calc_state["expression"] = ""
    calc_state["currentFunction"] = ""
    calc_state["result"] = ""
    update_display()

def backspace(b=None):
    calc_state["expression"] = calc_state["expression"][:-1]
    update_display()

def set_function(func):
    calc_state["currentFunction"] = func
    calc_state["expression"] = ""
    calc_state["result"] = ""
    update_display()

def calculate(b=None):
    try:
        full_expr = f"{calc_state['currentFunction']}({calc_state['expression']})" if calc_state["currentFunction"] else calc_state["expression"]

        if calc_state["currentFunction"]:
            value = float(calc_state["expression"]) if calc_state["expression"] else 0
            func = calc_state["currentFunction"]
            if func == "sin":
                calc_state["result"] = str(math.sin(math.radians(value)))
            elif func == "log":
                calc_state["result"] = str(math.log10(value)) if value > 0 else "Error"
            calc_state["currentFunction"] = ""  # Reset function mode
        else:
            calc_state["result"] = str(eval(calc_state["expression"]))

        # Keep input display unchanged
        calc_state["expression"] = full_expr

        update_display()
    except Exception:
        calc_state["result"] = "Error"
        update_display()

# --- Create Buttons ---
btn_clear = widgets.Button(description="C", button_style="warning")
btn_back = widgets.Button(description="⌫", button_style="warning")
btn_sin = widgets.Button(description="sin", button_style="info")
btn_log = widgets.Button(description="log", button_style="info")

btn_clear.on_click(clear_calc)
btn_back.on_click(backspace)
btn_sin.on_click(lambda b: set_function("sin"))
btn_log.on_click(lambda b: set_function("log"))

top_buttons = widgets.HBox([btn_clear, btn_back, btn_sin, btn_log])

# Bottom grid: numbers and basic operators
button_descriptions = [
    "7", "8", "9", "/",
    "4", "5", "6", "*",
    "1", "2", "3", "-",
    "0", ".", "=", "+"
]

num_buttons = []
for desc in button_descriptions:
    btn = widgets.Button(description=desc)
    if desc == "=":
        btn.on_click(calculate)
    elif desc in ["/", "*", "-", "+"]:
        btn.on_click(lambda b, d=desc: append_to_expr(d))
    else:
        btn.on_click(lambda b, d=desc: append_to_expr(d))
    num_buttons.append(btn)

grid = widgets.GridBox(num_buttons,
                         layout=widgets.Layout(
                             grid_template_columns="repeat(4, 1fr)",
                             grid_gap="5px"
                         ))

# --- Assemble the UI ---
calc_ui = widgets.VBox([expression_display, result_display, top_buttons, grid])
update_display()
display(calc_ui)
